# Creation of ground truth masks from COCO json file

In the following notebook we process the output of the label-studio api that has been used to manually annotate each image. 
The goal is to obtain two folders: images and labels.

In the labels directory are stored the label masks in png format.
Each mask has the same name as the image it refers to.


## Preliminary steps

In [1]:
import os
from pycocotools.coco import COCO
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt 

#create the notebook in the directory created by label-studio
cdir=os.getcwd()

print(cdir)

im_dir = cdir+'/images'
im_list = os.listdir(im_dir)
print(os.listdir(cdir))

#the annotation file exported from label-studio
ann_file = cdir+'/result.json'

#initialize coco api for annotations
coco=COCO(ann_file)



/Users/giacomozema/Desktop/industrial quality control of packages/top_and_side
['.DS_Store', 'result.json', 'images', '.ipynb_checkpoints', 'target_creation.ipynb']
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


In [2]:
#create a new directory for the label masks 

os.mkdir(path=cdir+'/labels')

## Check data integrity

In order to verify the integrity of the annotations we check the categories present in the annotations, we expect there to be only one category named 'package'

In [3]:
#get the names of the categories, useful when there is the need to select only a subset of classes

cats = coco.loadCats(coco.getCatIds())

names = [cat['name'] for cat in cats]

print(names)

['package']


The images Ids are retrieved from the coco json file. 

These ids are used to retrieve an image and its annotation mask

In [4]:
#get the image ids 

imgIds = coco.getImgIds()

#there are 200 images : ids 0-199
print(imgIds)


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221,

In [5]:
#get the ids from the images ids

annIds = coco.getAnnIds(imgIds)

print(annIds)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221,

## Creation of the masks and saving

In this final code cell, we perform the following operations:

- iterate over image IDs
- retrieve the image name for each ID
- get the annotation ID for that image
- get the mask for that annotation
- save the mask as .png in the labels folder with tha same name as the image preceded by "gt_"

In [6]:
images = coco.loadImgs(imgIds)

for im in imgIds:
    name = images[im]['file_name'].split("/")[-1]
    annId = coco.getAnnIds(im)[0]
    ann = coco.loadAnns(annId)[0]
    mask = coco.annToMask(ann)
    png = Image.fromarray(mask*255)
    png.save(cdir+'/labels/'+name)
    
                      